# News Crawler Application
*This is an interactive Jupyter Notebook which retrieves top news about a company (as asked by the user). Powered by News API*

This project was imagined after Facebook's stock price **dropped about 40%** between the closing on July 25, 2018 and opening of July 26, 2018. One of the main factors in the sudden drop in facebook's stock price was it's earning report missing expectations on revenue and showing a slowing user growth. This project is an attempt to help an investor keep track of big news about companies in an attempt to protect them from such a loss.

The following describes the workflow behind the application and how it captures major new sources at any given point of time.

In [23]:
# All the imports

# For user input
from ipywidgets import widgets
from ipywidgets import *
from traitlets import *
from IPython.display import display, Markdown

#For converting unicode to string
import unicodedata

#NewsApiClient to find URLs of all the trending news
from newsapi import NewsApiClient

# Requests is used to get an html markup from a URL
import requests

# Beautiful Soup for crawling and capturing text
from bs4 import BeautifulSoup

### Ask user about their choice of company

We'll start by asking the user about what company they're interested in. Run the cell and enter a name of a company and press _enter_

In [2]:
# Get user choice about company
text_input = widgets.Text()
print("Enter name of Company you want to find news about: ")
display(text_input)

# Confirm user choice
def handle_submit(sender):
    print "Alright, let's show you news about", text_input.value
    
text_input.on_submit(handle_submit)

Enter name of Company you want to find news about: 


Text(value=u'')

Alright, let's show you news about facebook


Store user's company choice for use in the rest of the notebook. (Note: need to convert from unicode to string for ease of use later)

In [3]:
# Convert user's company preference into a string ready for API query
company_name = unicodedata.normalize('NFKD', text_input.value).encode('ascii','ignore')

### Find top news article URLs

Next, we'll use the **News API** to find the top trending news about the company from CNBC and BBC News (in order to find a diversified, yet reputable mix of news articles). Currently we're only fetching 5 news articles, but that value can be changed.

In [72]:
#Call to NewsApi
newsapi = NewsApiClient(api_key='6ddbec75b59a4fabab49e2eafeba6df7')
all_articles = newsapi.get_everything(q = company_name,
                                      language='en',
                                      sources='cnbc',
                                      sort_by='relevancy',
                                      page_size = 5)

In [73]:
#Create a list of URLs to crawl through from the API
list_url = []
for item in all_articles['articles']:
    list_url.append(unicodedata.normalize('NFKD', item['url']).encode('ascii','ignore'))

### Iterate through URLs and capture text

Here we'll utilize the beautiful soup library to parse through html content and store only the article content. For each url:
1. We fetch the HTML
2. Parse the HTML document using Beautiful Soup 
3. Find and store the content of news article inside the webpage

In [74]:
# Iterate through list of urls, capture text on each and make presentable
article_content = []
for i in list_url:
    page = requests.get(i)
    soup = BeautifulSoup(page.content, 'html.parser')
    content = soup.find_all(['p'])
    for line in content:
        line_string = unicodedata.normalize('NFKD', line.get_text()).encode('ascii','ignore')
        if (len(line_string)>20):
            article_content.append(line_string)
    article_content.append(',')

### Display the articles

Use the Display class to present the articles in Markdown

In [75]:
counter = 1
display(Markdown("# Article " + str(counter)))
for line in article_content:
    if(line == ','):
        counter = counter + 1
        if (not(counter<=5)):
            break;
        display(Markdown("# Article " + str(counter)))
        continue
    display(Markdown(line))


# Article 1

Facebook is now offering some mobile app users a wireless-networking app without first disclosing that it's owned by Facebook, or that it collects information for the social networking company.

The app, Onavo Protect, provides users with a virtual private network, or VPN. Typically, a VPN cloaks the user's identity and adds other security features, making it a more secure way to get online, particularly when using public Wi-Fi networks.

Yet the Onavo app also tracks data that it shares with Facebook and others, "including the applications installed on your device, your use of those applications, the websites you visit and the amount of data you use," according to its own privacy policies.

Facebook can use that data to track what users do online even when they're not on one of its websites. The company could also find out how apps made by its rivals, such as Snap and Twitter, are being used.

...

Facebook has said it is doubling its staff that's focused on sensitive security and community issues to 20,000 by the end of next year, an investment that will impact its profits. 

Facebook and Google were not immediately available to comment on McNamee's remarks.

"They didn't [build their platform] because they wanted to blow up democracy. I don't think anyone thought they would be as successful a company as they are," McNamee said. "The reality is, though, they have been that successful."

But McNamee said demands from consumers would be more powerful than just trying to undermine Facebook's profits. He said consumers should push to own their own data and that technology companies should clarify how much of consumers' data they can really view.

"Did you sign up at the beginning to have them use your information for the rest of your life? To sell it to people for uses other than Facebook?" McNamee asked. "And for them to look for pictures of you everywhere they can find them and identify everything you're doing? Did you actually sign up for that? Do you even know  does any of us know   what's in the user agreement?"